In [13]:
import numpy as np
import pandas as pd 
import re 
from subprocess import check_output

In [19]:
!pip install keras


    100% |████████████████████████████████| 317kB 3.3MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 14.5MB/s ta 0:00:01
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics

/home/bardia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:

def clean_str(string):
    
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()



In [15]:
df = pd.read_csv("spam.csv" ,encoding='latin-1')

In [16]:
print(df.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [42]:
label = df.v1
texts = df.v2

In [43]:
test_label = label[5000:]
test_texts = texts[5000:]

In [44]:
label = label[:5000]
texts = texts[:5000]

In [46]:
test_label.shape

(572,)

In [47]:
le = LabelEncoder()

In [48]:
label = le.fit_transform(label)

In [49]:
label

array([0, 0, 1, ..., 0, 0, 0])

In [50]:
token = Tokenizer(num_words=1000)


In [51]:
token

In [52]:
token.fit_on_texts(texts)

In [53]:
mat_texts = token.texts_to_matrix(texts, mode = 'count')

In [54]:
print(mat_texts[:5])
print(label.shape,mat_texts.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 3. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]]
(5000,) (5000, 1000)


In [55]:
max_len = 100
cnn_texts_seq = token.texts_to_sequences(texts)

cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)


In [56]:
def check_model(model,x,y):
    model.fit(x,y,batch_size=32,epochs=10,verbose=1,validation_split=0.2)

In [57]:
def get_cnn_model_v3():    # added filter
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(256, #!!!!!!!!!!!!!!!!!!!
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

m = get_cnn_model_v3()
check_model(m,cnn_texts_mat,label)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 20)           20000     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 20)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 98, 256)           15616     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0         
__________

In [64]:
max_len = 100

cnn_texts_seq = token.texts_to_sequences(test_texts)

cnn_texts_mat2 = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)

le = LabelEncoder()
test_label = le.fit_transform(test_label)


In [65]:
score = m.evaluate(cnn_texts_mat2, test_label)

572/572 [==============================] - 0s 313us/step


In [66]:
score

[0.10640611497105679, 0.9790209790209791, 0.9790209790209791]

In [67]:
m.metrics_names

['loss', 'acc', 'binary_accuracy']